In [1]:
import pygplates
#import sphere_interp_kdtree as sphi
import healpy as hp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import os

%matplotlib inline

#rotation_filename = '/Users/Simon/GIT/pygplates-alpha/tutorials-sv/Data/Seton_etal_ESR2012_2012.1.rot'
rotation_filename = '/Users/wenchaocao/Desktop/New_Rotation.rot'
rotation_model = pygplates.RotationModel(rotation_filename)

### Make a multipoint feature (using healpix)
### with evely points distributed points on the sphere
# Note the number here controls resolution: 
# higher number--> higher resolution + slower code execution
nSide = 128
othetas,ophis = hp.pix2ang(nSide,np.arange(12*nSide**2))
othetas = np.pi/2-othetas
ophis[ophis>np.pi] -= np.pi*2
lats = np.degrees(othetas) 
lons = np.degrees(ophis)

multi_point_features = []
multi_point_feature = pygplates.Feature()
multi_point = pygplates.MultiPointOnSphere(zip(lats,lons))  
multi_point_feature.set_geometry(multi_point)
multi_point_feature.set_shapefile_attribute('Layer','do')
multi_point_features.append(multi_point)

mpf = pygplates.FeatureCollection(multi_point_feature)

# save the number of points covering the entire Earth to a variable,
# so that we can use it later to determine the ratio of Earth's surface
# covered by different paleo-environments
num_points_on_globe = lats.shape[0]
print num_points_on_globe

#plot all points on the sphere
#m = Basemap(projection='moll', lon_0=0, resolution='c')
#x,y = m(lons, lats)
#m.plot(x, y,'green', markersize=2)
#plt.show()


196608


In [3]:
def PaleogeogAreas(rotation_model,reconstruction_time,icesheet,mountain,landmass,shallow_marine):
# This function takes filenames for polygon files defining four different paleo-environment
# classes, and a multi-point feature with points covering the Earth's surface (presumably
# generated by healpix to ensure equal-area distribution, though they don't have to be)
# The function maps the paleo-environment attributes from the polygons onto the points
# (as shapefile_attributes) using pygplates plate partioning, and returns the cookie-cut
# points
    
    paleogeog_polygons = []

    if os.path.isfile(icesheet):
        features = pygplates.FeatureCollection(icesheet)
        for feature in features:
            feature.set_shapefile_attribute('Layer','i')
            paleogeog_polygons.append(feature)
    else:
        print 'No ice sheet found'

    features = pygplates.FeatureCollection(mountain)
    for feature in features:
        feature.set_shapefile_attribute('Layer','m')
        paleogeog_polygons.append(feature)

    features = pygplates.FeatureCollection(landmass)
    for feature in features:
        feature.set_shapefile_attribute('Layer','lm')
        paleogeog_polygons.append(feature)

    features = pygplates.FeatureCollection(shallow_marine)
    for feature in features:
        feature.set_shapefile_attribute('Layer','sm')
        paleogeog_polygons.append(feature)

    plate_partitioner = pygplates.PlatePartitioner(paleogeog_polygons,rotation_model,reconstruction_time=reconstruction_time,
                                                  sort_partitioning_plates=None)

    cookie_cut_points = plate_partitioner.partition_features(mpf,
                                                            properties_to_copy=[pygplates.PropertyName.gpml_shapefile_attributes])

    return cookie_cut_points


def PlotPaleogeography(cookie_cut_points,filename):
# This function takes cookie-cut points that are assumed evenly distributed across the sphere,
# with different attributes assigned to define the paleo-environment at each point.
# The points are plotted on the globe with different colours, and we count up
# the total number of points within each paleo-environment class. These numbers 
# are returned, such that they can be used to determine the ratio if the Earth's
# surface within each class (by dividing by total number of points covering Earth 
# at the original healpix resolution)

    plt.figure(figsize=(16,8))
    
    landmass_point_count = 0.
    mountain_point_count = 0.
    icesheet_point_count = 0.
    shallow_marine_point_count = 0.
    
    m = Basemap(projection='moll', lon_0=0, resolution='c')

    for feature in cookie_cut_points:
        if feature.get_shapefile_attribute('Layer')=='lm':
            for geometry in feature.get_all_geometries():
                x,y = m(geometry.to_lat_lon_array()[:,1],geometry.to_lat_lon_array()[:,0])
                m.plot(x,y,'.',color='yellow')
                landmass_point_count += len(x)
    for feature in cookie_cut_points:
        if feature.get_shapefile_attribute('Layer')=='sm':
            for geometry in feature.get_all_geometries():
                x,y = m(geometry.to_lat_lon_array()[:,1],geometry.to_lat_lon_array()[:,0])
                m.plot(x,y,'.',color='lightblue')
                shallow_marine_point_count += len(x)
    for feature in cookie_cut_points:
        if feature.get_shapefile_attribute('Layer')=='m':
            for geometry in feature.get_all_geometries():
                x,y = m(geometry.to_lat_lon_array()[:,1],geometry.to_lat_lon_array()[:,0])
                m.plot(x,y,'.',color='orange')
                mountain_point_count += len(x)
    for feature in cookie_cut_points:
        if feature.get_shapefile_attribute('Layer')=='i':
            for geometry in feature.get_all_geometries():
                x,y = m(geometry.to_lat_lon_array()[:,1],geometry.to_lat_lon_array()[:,0])
                m.plot(x,y,'.',color='purple')
                icesheet_point_count += len(x)
                
    plt.savefig(filename)
    plt.close()
    
    return landmass_point_count,mountain_point_count,icesheet_point_count,shallow_marine_point_count


### 1.Calculate paleogeography area before gaps fixed, before modified and after modified

In [22]:
Data = pd.read_csv('/Users/wenchaocao/Research/5_ConsistencyRatios_Fossils_Paleogeog/ReconPaleogeog_EB2015v1_AfterModifiedBasedOnFossils_410_2Ma/ReconPaleogeog_EB2015v1_AfterModifiedOnFossils_TestByFossils_410_2Ma.csv', delimiter=',')  
reconstruction_time_list = Data['Time intervals']
FigNum_list = np.arange(64,17,-2)
FromAge_list = Data['From_age']
ToAge_list = Data['To_age']

reconstruction_time = []
FigNum = []
FromAge= []
ToAge= []

for i,reconstruction_time,FigNum,FromAge,ToAge in zip(np.arange(1,25,1),reconstruction_time_list,FigNum_list,FromAge_list,ToAge_list):
    print i,reconstruction_time,FigNum,FromAge,ToAge
    
    # Paleogeog before modified
    #basedir = '/Users/wenchaocao/Research/6_CookieCutting_Corrections/'+str(reconstruction_time)+'/7_2_Reconstructed_Paleogeog_Matthews2016_'+str(reconstruction_time)+'Ma_GapsFixed/'
    
    #paleogeog after modified     
    #basedir = '/Users/wenchaocao/Research/6_CookieCutting_Corrections/'+str(reconstruction_time)+'/15_Reconstructed_Paleogeog_Matthews2016_'+str(reconstruction_time)+'Ma_Corrected/'
    
    # paleogeog before gaps fixed
    #basedir = '/Users/wenchaocao/Research/6_CookieCutting_Corrections/'+str(reconstruction_time)+'/6_2_Reconstructed_Paleogeog_Matthews2016_'+str(reconstruction_time)+'Ma/'
    
    # original paleogeog of Golonka
    basedir = '/Users/wenchaocao/Research/6_CookieCutting_Corrections/'+str(reconstruction_time)+'/6_Reconstructed_Paleogeog_EBIDs_'+str(reconstruction_time)+'Ma/'
    
    
    icesheet =  '%s/i_fig%d_%d_%d_reconstructed_%0.2fMa.shp' % (basedir,FigNum,FromAge,ToAge,reconstruction_time)
    mountain =  '%s/m_fig%d_%d_%d_reconstructed_%0.2fMa.shp' % (basedir,FigNum,FromAge,ToAge,reconstruction_time)
    landmass = '%s/lm_fig%d_%d_%d_reconstructed_%0.2fMa.shp' % (basedir,FigNum,FromAge,ToAge,reconstruction_time)
    shallow_marine = '%s/sm_fig%d_%d_%d_reconstructed_%0.2fMa.shp' % (basedir,FigNum,FromAge,ToAge,reconstruction_time)
    
    cookie_cut_points = PaleogeogAreas(rotation_model,reconstruction_time,icesheet,mountain,landmass,shallow_marine)
    lmpc,mpc,ipc,smpc = PlotPaleogeography(cookie_cut_points,'paleogeog_equal_area_points_%0.2f.png' % reconstruction_time)

    #tmp = pygplates.FeatureCollection(cookie_cut_points)
    #tmp.write('paleogeog_equal_area_points_22.00_nSide128.shp')
    print 'Time steps = %d' % reconstruction_time 
    print 'icesheet = %0.4f%%' % (100*ipc/num_points_on_globe)
    print 'mountain = %0.4f%%' % (100*mpc/num_points_on_globe)
    print 'landmass = %0.4f%%' % (100*lmpc/num_points_on_globe)
    print 'shallow_marine = %0.4f%%' % (100*smpc/num_points_on_globe)
    print 'deep_ocean = %0.4f%%' %(100-(100*ipc/num_points_on_globe)-(100*mpc/num_points_on_globe)-(100*lmpc/num_points_on_globe)-(100*smpc/num_points_on_globe))

    #print 'Print Percents %d Ma' % reconstruction_time
    #print 'icesheet area = %0.4f sq.km' % (EarthArea*(ipc/num_points_on_globe)/(1e7))
    #print 'mountain area = %0.4f sq.km' % (EarthArea*(mpc/num_points_on_globe)/(1e7))
    #print 'Landmass area = %0.4f sq.km' % (EarthArea*(lmpc/num_points_on_globe)/(1e7))
    #print 'shallow_marine area = %0.4f sq.km' % (EarthArea*(smpc/num_points_on_globe)/(1e7))
    #print 'deep_ocean area = %0.4f sq.km' % ((EarthArea/(1e7))-(EarthArea*(ipc/num_points_on_globe)/(1e7))-(EarthArea*(mpc/num_points_on_globe)/(1e7))-(EarthArea*(lmpc/num_points_on_globe)/(1e7))-(EarthArea*(smpc/num_points_on_globe)/(1e7)))
    #print 'Earth area= %0.2f sq.km' % (EarthArea/(1e7))
    EarthArea = 4.*np.pi*pygplates.Earth.mean_radius_in_kms**2
    #EarthArea = 510100000.
    #print 'EarthArea = %0.4f sq.km' % EarthArea
    print ''


1 6 64 11 2
Time steps = 6
icesheet = 1.7863%
mountain = 6.0135%
landmass = 22.0820%
shallow_marine = 10.9940%
deep_ocean = 59.1242%

2 14 62 20 11
Time steps = 14
icesheet = 1.4603%
mountain = 6.7510%
landmass = 20.7199%
shallow_marine = 12.9506%
deep_ocean = 58.1182%

3 22 60 29 20
Time steps = 22
icesheet = 1.4282%
mountain = 4.8269%
landmass = 23.5931%
shallow_marine = 13.6159%
deep_ocean = 56.5358%

4 33 58 37 29
Time steps = 33
icesheet = 1.0386%
mountain = 5.0013%
landmass = 22.5845%
shallow_marine = 14.3473%
deep_ocean = 57.0282%

5 45 56 49 37
No ice sheet found
Time steps = 45
icesheet = 0.0000%
mountain = 5.2561%
landmass = 22.0500%
shallow_marine = 14.4313%
deep_ocean = 58.2626%

6 53 54 58 49
No ice sheet found
Time steps = 53
icesheet = 0.0000%
mountain = 5.1366%
landmass = 22.3623%
shallow_marine = 14.8748%
deep_ocean = 57.6263%

7 76 52 81 58
Time steps = 76
icesheet = 0.2706%
mountain = 5.7922%
landmass = 19.3715%
shallow_marine = 14.1495%
deep_ocean = 60.4162%

8 90 5